In [2]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

In [4]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_cortadas"

Obtenemos de las carpetas originales las fotos

In [5]:
from PCA_funciones import matriz_fotos_desde_carpeta


#obtiene imagenes vectorizadas 30*30 y el nombre de la persona de cada foto
image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas) 

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


## Aplicamos Backpropagation
Transformamos las imagenes, con la cantidad de copoenentes 

### Escalamos y reducimos con PCA las imagenes

In [6]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" 
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

from PCA_funciones import transformar_imagen_pca

"""# Función para transformar una imagen con un rango específico de componentes PCA
def transformar_imagen_pca(imagen, scaler, pca_components, num_componentes=50, inicio_componente=0):
    # Estandarizar la imagen
    imagen_estandarizada = scaler.transform(imagen)
    
    # Seleccionar el rango de componentes
    componentes_seleccionados = pca_components[inicio_componente:inicio_componente + num_componentes]
    
    # Realizar la proyección manualmente
    imagen_pca = np.dot(imagen_estandarizada, componentes_seleccionados.T)
    return imagen_pca
"""

# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)


[[ 7.22160860e+00 -7.80849051e+00 -2.07612021e-01 ... -7.16897218e-01
  -2.28025618e-01 -3.38966655e-01]
 [-7.84286624e+00 -1.12024841e+01  4.89598300e-01 ... -7.43509551e-01
  -6.18654161e-01  2.88035939e-01]
 [-6.55630242e+00 -7.80088794e+00 -1.49179444e+00 ... -1.37140978e+00
  -2.96450608e-01  2.05965925e-02]
 ...
 [ 3.55657307e+00  6.49301206e-01 -6.23891347e+00 ... -6.32500626e-02
   3.18870940e-01 -6.26304974e-01]
 [-2.65755973e+00 -4.44534213e-03 -8.82886566e+00 ...  2.92722209e-01
  -9.78650328e-02 -2.65089331e-01]
 [-2.62864833e+00  8.66772857e-01 -8.11920552e+00 ...  1.20877517e-01
  -1.40180893e-02 -1.42284246e-01]]


In [7]:
print("shape reducido " , imagenes_pca_reducidas.shape) # la segunda cantidad indica el numero de componentes una vez reducido
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print("Cantidad de Personas: " , cantidad_personas) 
cantidad_fotos = len(image_person)
print("Cantidad Total de Fotos: " , cantidad_fotos)


shape reducido  (1206, 60)
Cantidad de Personas:  19
Cantidad Total de Fotos:  1206


## Back Propagation

### Transformamos los nombres de las personas a vector de salida

In [8]:
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
print(salida_lista)
salida_lista.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(1206, 19)

### Inicializamos parametros

In [9]:
# el vector de entrada X
X=imagenes_pca_reducidas
X.shape

X = imagenes_pca_reducidas
Y = salida_lista

# Convierte la lista a un arreglo NumPy
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 30

Separamos conjunto de datos de entrenamiento y test

In [25]:

# Semilla para reproducibilidad
#np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
#tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
np.random.seed(97) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(97) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.

from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

## Back propagation manual

In [26]:
print(len(X))
print(len(X_train))
print(len(X_test))

1206
964
242


In [27]:
print("X_train shape: ",X_train.shape)
print("y_train shape: ",Y_train.shape)

X_train shape:  (964, 60)
y_train shape:  (964, 19)


#Aca estamos trabajando 

In [34]:
# 
# lo hago con  matrices de pesos
# puedo tener tantos inputs como quiera
# puedo tener tantas neuronas ocultas como quiera
# puedo tener tanas neuronas de salida como quiera
# fuera de este codigo esta la decision que tomo segun el valor de salida de cada neurona de salida

import math
import numpy as np
import matplotlib.pyplot as plt

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
            
    
        case "softmax":
            exps = np.exp(x - np.max(x, axis=0, keepdims=True))  # Estabilización numérica
            y = exps / np.sum(exps, axis=0, keepdims=True)
        
    return y

func_eval_vec = np.vectorize(func_eval)

def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
        case "softmax":
            d = y * (1.0 - y)  #La derivada de softmax es más compleja y normalmente se calcula usando la matriz Jacobiana. 
                                # En este ejemplo, hemos usado una simplificación y * (1.0 - y) que es una aproximación y 
                                # puede no ser precisa en todos los casos.            
    return d

deriv_eval_vec = np.vectorize(deriv_eval)

#new_faces_array_transformed.shape #vemos el vector con las caras y sus 60 componentes a partir de la 3ra

# Normalización de las entradas
#mean = np.mean(X, axis=0)
#std = np.std(X, axis=0)
#X = (X - mean) / std

# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
#salida = np.array(salida_lista)
#Y = salida #salida.reshape(len(X),1)

filas_qty = len(X_train)        # la cantidad de fotos
input_size = X_train.shape[1]   # entradas la cantidad de componentes
                                # hidden_size = 3  # neuronas capa oculta
hidden_size1 = filas_qty        # Neuronas en la primera capa oculta, arrancamos probando con la cantidad de fotos
hidden_size2 = 3                # Neuronas en la segunda capa oculta

output_size = cantidad_personas # vector con la probabilidad q sea la persona de la posicion

# defino las funciones de activacion de cada capa
hidden_FUNC = 'logsig'  # probamos con  la logistica para las capas ocultas
output_FUNC =  'purelin' #'logsig' #'tansig'  # probamos con softmax porque es un probema de clasificación

'''
Si tu problema es de clasificación multiclase y estás utilizando softmax como función de activación para la capa de salida,
sí, las salidas deben sumar 1 porque softmax convierte el vector de salida en una distribución de probabilidad.
Si estás resolviendo un problema de regresión o estás utilizando una función de activación diferente (como sigmoid, tansig), 
no es necesario que las salidas sumen 1.
'''

# Incializo las matrices de pesos azarosamente
# W1 son los pesos que van del input a la capa oculta
# W2 son los pesos que van de la capa oculta a la segunda capa oculta
# W3 son los pesos que van de la 2da Capa Oculta a la salida

np.random.seed(1021) # random seed para que las corridas sean reproducibles
W1 = np.random.uniform(-0.15, 0.15, [hidden_size1, input_size])
X01 = np.random.uniform(-0.15, 0.15, [hidden_size1, 1] )
W2 = np.random.uniform(-0.150, 0.15, [hidden_size2, hidden_size1])
X02 = np.random.uniform(-0.15, 0.15, [hidden_size2, 1] )
W3 = np.random.uniform(-0.15, 0.15, [output_size, hidden_size2])
X03 = np.random.uniform(-0.15, 0.15, [output_size, 1] )


# # Avanzo la red, forward
# # para TODOS los X al mismo tiempo ! 
# #  @ hace el producto de una matrix por un vector_columna
# hidden_estimulos = W1 @ X.T + X01
# hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
# output_estimulos = W2 @ hidden_salidas + X02
# output_salidas = func_eval_vec(output_FUNC, output_estimulos)


# Función de propagación hacia adelante (forward propagation)
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
    hidden1_estimulos = W1 @ X.T + b1
    hidden1_salidas = func_eval_vec(hidden_FUNC, hidden1_estimulos)
    hidden2_estimulos = W2 @ hidden1_salidas + b2
    hidden2_salidas = func_eval_vec(hidden_FUNC, hidden2_estimulos)
    output_estimulos = W3 @ hidden2_salidas + b3
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)
 #   print("output_estimulos: ",output_estimulos)
    return hidden1_salidas, hidden2_salidas, output_salidas, hidden1_estimulos, hidden2_estimulos, output_estimulos


hidden_salida1, hidden_salida2, output_salidas, _, _ ,_ = forward_propagation(X_train, W1, X01, W2, X02, W3, X03)


# calculo el error promedi general de TODOS los X
Error= np.mean( (Y_train.T - output_salidas)**2 )

# Inicializo
#epoch_limit = 5000    # para terminar si no converge
Error_umbral = 1.0e-06
#Error_umbral = 1.0e-15
learning_rate = 0.5
#learning_rate = 0.2
Error_last = 10    # lo debo poner algo dist a 0 la primera vez
epoch = 0

while ( math.fabs(Error_last-Error)>Error_umbral and (epoch < epoch_limit)):
    epoch += 1
    Error_last = Error

    # recorro siempre TODA la entrada
    for fila in range(filas_qty): #para cada input x_sub_fila del vector X
        # propagar el x hacia adelante
        # hidden_estimulos = W1 @ X[fila:fila+1, :].T + X01
        # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
        # output_estimulos = W2 @ hidden_salidas + X02
        # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
        hidden_salida1, hidden_salida2, output_salidas,  _, _ ,_  = forward_propagation(X_train[fila:fila+1, :], W1, X01, W2, X02, W3, X03)

        # calculo los errores en la capa hidden y la capa output
        ErrorSalida = Y_train[fila:fila+1,:].T - output_salidas
        # output_delta es un solo numero
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        
        # hidden_delta es un vector columna
        # hidden_delta = deriv_eval_vec(hidden_FUNC, hidden_salidas)*(W2.T @ output_delta)
        # hidden_delta es un vector columna
        hidden_delta2 = deriv_eval_vec(hidden_FUNC, hidden_salida2)*(W3.T @ output_delta)

        # hidden_delta es un vector columna
        hidden_delta1 = deriv_eval_vec(hidden_FUNC, hidden_salida1)*(W2.T @ hidden_delta2)

        # ya tengo los errores que comete cada capa
        # corregir matrices de pesos, voy hacia atras
        # backpropagation
        W1 = W1 + learning_rate * (hidden_delta1 @ X_train[fila:fila+1, :] )
        X01 = X01 + learning_rate * hidden_delta1
        W2 = W2 + learning_rate * (hidden_delta2 @ hidden_salida1.T )
        X02 = X02 + learning_rate * hidden_delta2
        W3 = W3 + learning_rate * (output_delta @ hidden_salida2.T)
        X03 = X03 + learning_rate * output_delta

    # ya recalcule las matrices de pesos
    # ahora avanzo la red, feed-forward
    # hidden_estimulos = W1 @ X.T + X01
    # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
    # output_estimulos = W2 @ hidden_salidas + X02
    # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    
    hidden_salida1, hidden_salida2, output_salidas,  hidden_estimulo1, hidden_estimulo2 , hidden_ouput  = forward_propagation(X_train[fila:fila+1, :], W1, X01, W2, X02, W3, X03)
    

    # calculo el error promedio general de TODOS los X
    Error= np.mean( (Y_train.T - output_salidas)**2 )

    # tengo que hacer X01.T[0]  para que pase el vector
   # grafico.graficarVarias(W1, X01.T[0], epoch, -1)


# Evaluar el modelo en el conjunto de prueba
# loss, accuracy = model.evaluate(X_test, y_test)
# print("Accuracy on test set:", accuracy)

# Hacer predicciones con el modelo entrenado
# predictions = model.predict(new_data)
hidden_salida1_modelo, hidden_salida2_modelo, output_salidas_modelo,  hidden_estimulo1_modelo, hidden_estimulo2_modelo , hidden_ouput_modelo  = forward_propagation(X_train, W1, X01, W2, X02, W3, X03)

Error

0.1005672935392694

In [30]:
hidden_salida1_modelo, hidden_salida2_modelo, output_salidas_modelo,  hidden_estimulo1_modelo, hidden_estimulo2_modelo , hidden_ouput_modelo  = forward_propagation(X_train, W1, X01, W2, X02, W3, X03)
    


In [35]:
print(output_salidas_modelo.shape)

salidas_train = output_salidas_modelo.T
salidas_train

(19, 964)


array([[ 3.80886764e-02,  1.32167553e-02, -1.75825870e-05, ...,
         2.22229314e-02,  1.08106101e-02,  2.44713188e-03],
       [ 3.80895789e-02,  1.32160271e-02, -1.65925619e-05, ...,
         2.22219843e-02,  1.08113776e-02,  2.44698771e-03],
       [ 3.80893885e-02,  1.32161809e-02, -1.68028808e-05, ...,
         2.22221843e-02,  1.08112155e-02,  2.44701816e-03],
       ...,
       [-1.10984670e-01, -1.19455759e-02,  1.00009221e+00, ...,
         1.96048500e-02, -1.32268999e-02,  2.29456145e-03],
       [ 3.80882735e-02,  1.32170803e-02, -1.80237248e-05, ...,
         2.22233541e-02,  1.08102676e-02,  2.44719622e-03],
       [ 5.46075888e-06, -6.01325518e-08, -5.30890644e-08, ...,
        -3.10996627e-06,  1.46206941e-07, -3.50790394e-07]])

In [32]:
for i in range(0,18):
    print(sum(salidas_train[i]))

19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0


### medimos

In [150]:
salidas_train.shape

(242, 19)

In [151]:
len(salidas_train)

242

In [155]:
def ConvertirSalidas(Y, salidas):
    caracteristicas_pred = []
    caracteristicas_reales=[]
    probabilidades_pred=[]
    # Iterar sobre las pred0icciones y mostrar el resultado para cada foto
    for i in range(len(salidas)):
        # Obtener la predicción para la i-ésima foto
        prediction = salidas[i]
        # Encontrar la etiqueta predicha (la clase con probabilidad más alta)
        predicted_label = np.argmax(prediction) # Identifica la clase que el modelo considera más probable para la imagen
        # Mostrar el resultado
        real = np.argmax(Y[i])
        predicted_probability= np.max(prediction)
        print(f"Foto {i+1}: Real = {real}, Predicción = {predicted_label}, Probabillidad = {predicted_probability}, Probabilidades = {prediction}") # nro de foto - la clase con mayor probabilidad - probabilidad de pertenencia a cada clase.
        
        caracteristicas_pred.append(predicted_label)
        caracteristicas_reales.append(real)
        probabilidades_pred.append(predicted_probability)
    return caracteristicas_pred, caracteristicas_reales, probabilidades_pred

In [156]:
ConvertirSalidas(Y_train, salidas_train)

Foto 1: Real = 5, Predicción = 2, Probabillidad = 0.08864560099313838, Probabilidades = [0.01759119 0.04747497 0.0886456  0.02549351 0.04242404 0.01011768
 0.0468228  0.04674432 0.0400899  0.06173655 0.05515728 0.08701904
 0.05478118 0.04169881 0.0614647  0.04346381 0.03299259 0.06097792
 0.04560966]
Foto 2: Real = 8, Predicción = 2, Probabillidad = 0.08864560189020171, Probabilidades = [0.01759119 0.04747497 0.0886456  0.02549351 0.04242404 0.01011768
 0.0468228  0.04674432 0.0400899  0.06173655 0.05515728 0.08701904
 0.05478118 0.04169881 0.0614647  0.04346381 0.03299259 0.06097792
 0.04560966]
Foto 3: Real = 18, Predicción = 2, Probabillidad = 0.08864562488620359, Probabilidades = [0.0175912  0.04747498 0.08864562 0.02549351 0.04242405 0.01011768
 0.04682281 0.04674433 0.04008991 0.06173656 0.0551573  0.08701907
 0.0547812  0.04169882 0.06146472 0.04346382 0.0329926  0.06097794
 0.04560967]
Foto 4: Real = 3, Predicción = 2, Probabillidad = 0.08864561069110387, Probabilidades = [0.01

([2,
  2,
  2,
  2,
  2,
  5,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  5,
  5,
  2,
  2,
  2,
  2,
  3,
  5,
  3,
  2,
  3,
  2,
  2,
  5,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  5,
  2,
  2,
  2,
  5,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  5,
  2,
  2,
  2,
  2,
  5,
  5,
  2,
  5,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  5,
  2,
  2,
  2,
  5,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,


### Guardamos el modelo entrenado

In [127]:
    ruta_modelo_3c = os.path.join(root_path , 'Back_propagation')
ruta_modelo_3c = os.path.join(ruta_modelo_3c , 'Modelo_3c')

print(ruta_modelo_3c)
# Guardar los pesos y biases en la carpeta especificada
np.save(os.path.join(ruta_modelo_3c, "W1.npy"), W1)
np.save(os.path.join(ruta_modelo_3c, "X01.npy"), X01)
np.save(os.path.join(ruta_modelo_3c, "W2.npy"), W2)
np.save(os.path.join(ruta_modelo_3c, "X02.npy"), X02)
np.save(os.path.join(ruta_modelo_3c, "W3.npy"), W3)
np.save(os.path.join(ruta_modelo_3c, "X03.npy"), X03)


../Back_propagation\Modelo_3c


## Obtenemos el modelo guardado y lo aplicamos a test

In [128]:
# Cargar los pesos y biases desde la carpeta especificada
W1_loaded = np.load(os.path.join(ruta_modelo_3c, "W1.npy"))
X01_loaded = np.load(os.path.join(ruta_modelo_3c, "X01.npy"))
W2_loaded = np.load(os.path.join(ruta_modelo_3c, "W2.npy"))
X02_loaded = np.load(os.path.join(ruta_modelo_3c, "X02.npy"))
W3_loaded = np.load(os.path.join(ruta_modelo_3c, "W3.npy"))
X03_loaded = np.load(os.path.join(ruta_modelo_3c, "X03.npy"))


In [129]:
# Aplicar los pesos cargados al conjunto de test
_, _, output_salidas_test, _, _, _ = forward_propagation(X_test, W1_loaded, X01_loaded, W2_loaded, X02_loaded, W3_loaded, X03_loaded)


In [138]:
print(output_salidas_test.shape)

saidas_test = output_salidas_test.T
saidas_test

(19, 242)


array([[0.01759119, 0.04747497, 0.0886456 , ..., 0.03299259, 0.06097792,
        0.04560966],
       [0.01759119, 0.04747497, 0.0886456 , ..., 0.03299259, 0.06097792,
        0.04560966],
       [0.0175912 , 0.04747498, 0.08864562, ..., 0.0329926 , 0.06097794,
        0.04560967],
       ...,
       [0.01759124, 0.04747489, 0.08864536, ..., 0.0329926 , 0.06097776,
        0.04560958],
       [0.01759143, 0.04747567, 0.088647  , ..., 0.03299303, 0.06097902,
        0.0456104 ],
       [0.01759125, 0.04747486, 0.08864527, ..., 0.0329926 , 0.0609777 ,
        0.04560955]])

In [140]:
for i in range(0,18):
    print(sum(saidas_test[i]))

0.9103055712655986
0.9103055808966067
0.9103057943334628
0.9103056618074359
0.9103017598487002
1.532996920494786
0.910305084668512
1.5848705839830148
0.9103102272628811
0.9103058532750377
0.9103124395526199
0.9103056002869003
0.9103055483967172
1.5329858546744035
1.5329680942688335
0.9105227952202181
0.9114229444191291
0.9103055129617834


# clasificamos las predicciones de test

In [148]:
# Iterar sobre las predicciones y mostrar el resultado para cada foto
for i in range(len(saidas_test.T)):
    # Obtener la predicción para la i-ésima foto
    prediction = saidas_test[i]
    # Encontrar la etiqueta predicha (la clase con probabilidad más alta)
    predicted_label = np.argmax(prediction) # Identifica la clase que el modelo considera más probable para la imagen
    # Mostrar el resultado
    real = np.argmax(Y_test[i])
    predicted_probability= np.max(prediction)
    print(f"Foto {i+1}: Real = {real}, Predicción = {predicted_label}, Probabillidad = {predicted_probability}, Probabilidades = {prediction}") # nro de foto - la clase con mayor probabilidad - probabilidad de pertenencia a cada clase.


Foto 1: Real = 4, Predicción = 2, Probabillidad = 0.08864560099313838, Probabilidades = [0.01759119 0.04747497 0.0886456  0.02549351 0.04242404 0.01011768
 0.0468228  0.04674432 0.0400899  0.06173655 0.05515728 0.08701904
 0.05478118 0.04169881 0.0614647  0.04346381 0.03299259 0.06097792
 0.04560966]
Foto 2: Real = 11, Predicción = 2, Probabillidad = 0.08864560189020171, Probabilidades = [0.01759119 0.04747497 0.0886456  0.02549351 0.04242404 0.01011768
 0.0468228  0.04674432 0.0400899  0.06173655 0.05515728 0.08701904
 0.05478118 0.04169881 0.0614647  0.04346381 0.03299259 0.06097792
 0.04560966]
Foto 3: Real = 18, Predicción = 2, Probabillidad = 0.08864562488620359, Probabilidades = [0.0175912  0.04747498 0.08864562 0.02549351 0.04242405 0.01011768
 0.04682281 0.04674433 0.04008991 0.06173656 0.0551573  0.08701907
 0.0547812  0.04169882 0.06146472 0.04346382 0.0329926  0.06097794
 0.04560967]
Foto 4: Real = 17, Predicción = 2, Probabillidad = 0.08864561069110387, Probabilidades = [0.

## Precision de datos de prueba

Usando esta matriz, las fórmulas serían:

__Precision:__ Proporción de TP sobre todos los predichos positivos (TP + FP).
__Recall:__ Proporción de TP sobre todos los reales positivos (TP + FN).
__F1 Score:__ Media armónica de precisión y recall.
__Support:__ Cantidad total de ejemplos reales positivos y negativos.

__Precision (Precisión)__
La proporción de verdaderos positivos (TP) entre todos los ejemplos que el modelo ha clasificado como positivos.
Cómo se calcula: 
precision  = TP / (TP + FP)
            donde TP son los verdaderos positivos y FP son los falsos positivos.
Uso: Indica cuán preciso es el modelo cuando predice la clase positiva. Es útil en situaciones donde el costo de un falso positivo es alto.


__Recall (Sensibilidad)__
La proporción de verdaderos positivos entre todos los ejemplos que realmente son positivos.
Cómo se calcula:
recall  = TP / (TP + FN)
        donde TP son los verdaderos positivos y FN son los falsos negativos.
*Uso:* Indica cuán bien el modelo captura los verdaderos positivos. Es útil en situaciones donde el costo de un falso negativo es alto.
__F1 Score__
Es una medida combinada que toma en cuenta tanto la precisión (precision) como la exhaustividad (recall).
Cómo se calcula: Es la media armónica de la precisión y el recall.
Fórmula:
 𝐹1 = 2 × precision × recall / ( precision + recall)
  
*Uso:* Se utiliza para evaluar modelos de clasificación, especialmente cuando hay un desequilibrio entre las clases.

__Support (Soporte)__
Qué mide: El número de ocurrencias de cada clase en el conjunto de datos verdadero.
*Uso:* Proporciona contexto para las métricas de precisión, recall y F1 score, mostrando cuántos ejemplos de cada clase hay en los datos.

In [143]:
Y_test.shape

(242, 19)

In [145]:
y_pred_test

array([[0.01759119, 0.04747497, 0.0886456 , ..., 0.03299259, 0.06097792,
        0.04560966],
       [0.01759119, 0.04747497, 0.0886456 , ..., 0.03299259, 0.06097792,
        0.04560966],
       [0.0175912 , 0.04747498, 0.08864562, ..., 0.0329926 , 0.06097794,
        0.04560967],
       ...,
       [0.01759124, 0.04747489, 0.08864536, ..., 0.0329926 , 0.06097776,
        0.04560958],
       [0.01759143, 0.04747567, 0.088647  , ..., 0.03299303, 0.06097902,
        0.0456104 ],
       [0.01759125, 0.04747486, 0.08864527, ..., 0.0329926 , 0.0609777 ,
        0.04560955]])

In [142]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Obtener las predicciones para el conjunto de prueba
y_pred_test = output_salidas_test.T

# Imprimir el reporte de clasificación
print(classification_report(Y_test, y_pred_test))

# Imprimir la matriz de confusión
print(confusion_matrix(Y_test, y_pred_test))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

## Nuevas Fotos

In [81]:
p_Fotos = root_path + "Nuevas_Fotos"
p_Fotos

'../Nuevas_Fotos'

In [39]:
from Funciones import  Leer_fotos
fotos=Leer_fotos(p_Fotos)
fotos

{'Abel': [array([[[201, 206, 215],
          [202, 207, 216],
          [203, 208, 217],
          ...,
          [105, 131, 167],
          [105, 131, 167],
          [105, 131, 167]],
  
         [[165, 174, 187],
          [164, 176, 188],
          [167, 176, 189],
          ...,
          [106, 132, 168],
          [106, 132, 168],
          [106, 132, 168]],
  
         [[116, 137, 158],
          [115, 139, 159],
          [117, 138, 159],
          ...,
          [107, 133, 169],
          [108, 134, 170],
          [108, 134, 170]],
  
         ...,
  
         [[103,  73,  14],
          [ 93,  64,   3],
          [ 87,  57,   0],
          ...,
          [ 52,  35,   2],
          [ 54,  37,   4],
          [ 57,  38,   3]],
  
         [[ 94,  66,   5],
          [ 91,  64,   0],
          [ 90,  62,   1],
          ...,
          [ 52,  33,   0],
          [ 55,  35,   0],
          [ 58,  39,   2]],
  
         [[ 94,  67,   3],
          [ 92,  65,   1],
          [ 91, 

In [12]:
# Mostrar información
nombres_personas = list(fotos.keys())
print("Cantidad de personas:", len(nombres_personas))
print("Cantidad de fotos por persona:")
for nombre_persona, fotos_persona in fotos.items():
    print(f"- {nombre_persona}: {len(fotos_persona)}")

Cantidad de personas: 15
Cantidad de fotos por persona:
- Abel: 1
- Carlos: 1
- Federico G: 1
- Federico R: 1
- Franco S: 1
- Gerard: 1
- Gustavo: 1
- Joaquin: 1
- Lautaro: 1
- Lisandro: 1
- Matias: 1
- Natalia: 1
- Noelia: 1
- Paola: 1
- Victorio: 1


In [28]:
def recortar_imagen_C(image):
    import os
    import cv2
   # import matplotlib.pyplot as plt
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

In [15]:
import os
import cv2
#import matplotlib.pyplot as plt

def recortar_imagen(image):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

In [18]:
def procesar_carpetas(directorio_origen, directorio_destino):
    # Iterar sobre las carpetas dentro del directorio

    # Crea el directorio si no existe
    if not os.path.exists(directorio_destino):
       os.makedirs(directorio_destino)

    for root, dirs, files in os.walk(directorio_origen):
        for dir_name in dirs:
            print("Carpeta:", dir_name)
            dir_path = os.path.join(root, dir_name) #directorio origen de la persona 
            dir_path_recorte = os.path.join(directorio_destino, dir_name) #directorio destino de la persona
            
            if not os.path.exists(dir_path_recorte):
                os.makedirs(dir_path_recorte) #creamos carpetas para cada nombre
            
            images = []

            numero=0 #numero de la foto dentro del directorio
            
            for file in glob.glob(os.path.join(dir_path, "*.jp*g")) :
                # Verificar si el archivo es una imagen
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    # Leer la imagen
                    #mage_path = os.path.join(dir_path, file)
                    image_path = file
                    print(image_path)
                    image = cv2.imread(image_path)
                    
                    if image is not None:
                        # Recortar la imagen
                        numero +=1
                        face_images = recortar_imagen(image)
                        images.extend(face_images)
                        numero_interno=0
                        for face_image in face_images:
                            gray_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                            numero_interno +=1
                            numero_formateado = str(numero).zfill(2)
                            numero_interno_formateado = str(numero_interno).zfill(2)
                            #_{image_path.split('.')[0]}_
                            output_path = os.path.join(dir_path_recorte , f"{dir_name}_{numero_formateado}_{numero_interno_formateado}.jpg")
                            # Save the cropped face to the output directory                       
                            cv2.imwrite(output_path, gray_image)



In [19]:
nuevas_recortadas = root_path + "nuevas_recortadas"
procesar_carpetas(p_Fotos, nuevas_recortadas)

Carpeta: Abel
../Nuevas_Fotos\Abel\Abel.jpeg
Carpeta: Carlos
../Nuevas_Fotos\Carlos\Carlos.jpeg
Carpeta: Federico G
../Nuevas_Fotos\Federico G\Federico G.jpeg
Carpeta: Federico R
../Nuevas_Fotos\Federico R\Federico R.jpeg
Carpeta: Franco S
../Nuevas_Fotos\Franco S\Franco S.jpeg
Carpeta: Gerard
../Nuevas_Fotos\Gerard\Gerard.jpeg
Carpeta: Gustavo
../Nuevas_Fotos\Gustavo\Gustavo.jpeg
Carpeta: Joaquin
../Nuevas_Fotos\Joaquin\Joaquin.jpeg
Carpeta: Lautaro
../Nuevas_Fotos\Lautaro\Lautaro.jpeg
Carpeta: Lisandro
../Nuevas_Fotos\Lisandro\Lisandro.jpeg
Carpeta: Matias
../Nuevas_Fotos\Matias\Matias.jpeg
Carpeta: Natalia
../Nuevas_Fotos\Natalia\Natalia.jpeg
Carpeta: Noelia
../Nuevas_Fotos\Noelia\noelia.jpeg
Carpeta: Paola
../Nuevas_Fotos\Paola\Paola.jpeg
Carpeta: Victorio
../Nuevas_Fotos\Victorio\Victorio.jpeg


In [20]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(nuevas_recortadas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


In [ ]:
num_componentes=60
inicio_componente
# Transformar la nueva imagen utilizando los componentes especificados
imagen_pca_reducida = transformar_imagen_pca(image_matrix, scaler, pca_components, num_componentes, inicio_componente)

imagen_pca_reducida.shape

In [ ]:


prediction = model.predict(imagen_pca_reducida)

print("Predicción:", prediction)